In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
import os

In [2]:
# mount google drive where preprocessed images are
# Mount drive and download data from EDA step 1
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# make sure these are consistent with the preprocessing operation
TARGET_SIZE = (192,256) # in preparation for CNN, From EDA, size in S1 is (450,600)
BATCH_SIZE = 32
RANDOM_SEED = 42

# configure google drive folders from EDA_Preprocessing.ipynb S1 file
GDIR_PROJECT = '/content/drive/MyDrive/AAI-521/Module7/TeamProject'
GDIR_PROJECT_EDA = GDIR_PROJECT + '/EDA_PrePro'
GDIR_PROJECT_EDA_S2 = GDIR_PROJECT_EDA + '/S2'
GDIR_PROJECT_EDA_S2_VAL = GDIR_PROJECT_EDA_S2 + '/val'
GDIR_PROJECT_EDA_S2_TRAIN = GDIR_PROJECT_EDA_S2 + '/train'

Mounted at /content/drive


In [3]:
!pwd

/content


In [4]:
# copy gdrive files to virtual environment folder
VDIR_PROJECT = '/content/skin_cancer/training'
os.makedirs(VDIR_PROJECT, exist_ok=True)


In [7]:
# copy zipped file to virtual
SOURCE_FILE = '/content/drive/MyDrive/AAI-521/Module7/TeamProject/EDA_PrePro/S2-20241201T123737Z-001.zip'

!cp -r {SOURCE_FILE} {VDIR_PROJECT}


In [8]:
# unzip S2
VIRTUAL_FILE = '/content/skin_cancer/training/S2-20241201T123737Z-001.zip'
!unzip {VIRTUAL_FILE} -d {VDIR_PROJECT}

Streaming output truncated to the last 5000 lines.
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1971.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1529.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1418.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1592.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1406.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1399.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1370.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1603.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1899.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1526.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_2010.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1354.jpg  
  inflating: /content/skin_cancer/training/S2/train/bkl/image_1386.jpg  


In [9]:


VDIR_PROJECT_S2 = VDIR_PROJECT + '/S2'
VDIR_PROJECT_S2_VAL = VDIR_PROJECT_S2 + '/val'
VDIR_PROJECT_S2_TRAIN = VDIR_PROJECT_S2 + '/train'

In [24]:
# create image iterators
datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values

# create train dataset generator
train_generator = datagen.flow_from_directory(
    VDIR_PROJECT_S2_TRAIN,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode = 'categorical',
    seed = RANDOM_SEED)


# create validation dataset generator
val_generator = datagen.flow_from_directory(
    VDIR_PROJECT_S2_VAL,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode = 'categorical',
    seed = RANDOM_SEED)



Found 37989 images belonging to 7 classes.
Found 1094 images belonging to 7 classes.


In [25]:
# compute training steps
GDIR_PROJECT_EDA_S1 = GDIR_PROJECT_EDA + '/S1'
GDIR_PROJECT_EDA_S1_VAL = GDIR_PROJECT_EDA_S1 + '/val'
GDIR_PROJECT_EDA_S1_TRAIN = GDIR_PROJECT_EDA_S1 + '/train'
# read train and val set metadata from S1
metadata_df_train = pd.read_csv(GDIR_PROJECT_EDA_S1_TRAIN + '/metadata_train.csv')
metadata_df_val = pd.read_csv(GDIR_PROJECT_EDA_S1_VAL + '/metadata_val.csv')

# get number of train and validation samples
orig_train_samples = len(metadata_df_train)
orig_val_samples = len(metadata_df_val)
print("ORIG TRAIN SAMPLES: ", orig_train_samples)
print("ORIG VAL SAMPLES: ", orig_val_samples)

train_steps = np.ceil(orig_train_samples / BATCH_SIZE).astype('uint32')

val_steps = np.ceil(orig_val_samples / BATCH_SIZE).astype('uint32')

print("TRAIN STEPS: ", train_steps)
print("VAL STEPS: ", val_steps)

ORIG TRAIN SAMPLES:  8864
ORIG VAL SAMPLES:  1094
TRAIN STEPS:  277
VAL STEPS:  35


In [26]:
len(train_generator)

1188

In [27]:
# create base classifier model
# Load the DenseNet model with pre-trained weights from ImageNet
input_shape = (TARGET_SIZE[0],TARGET_SIZE[1], 3)
print("input_shape = ", input_shape)
base_model = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model layers
base_model.trainable = False

# Create a new model on top of the base model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')  # Use 'softmax' for multi-class classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[categorical_accuracy])

input_shape =  (192, 256, 3)


In [15]:
base_model.summary()

Model: "densenet121"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 192, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_2          │ (None, 198, 262, 3)    │              0 │ input_layer_2[0][0]    │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 96, 128, 64)    │          9,408 │ zero_padding2d_2[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 96, 128, 64)    │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 96, 128, 64)    │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_3          │ (None, 98, 130, 64)    │              0 │ conv1_relu[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1 (MaxPooling2D)      │ (None, 48, 64, 64)     │              0 │ zero_padding2d_3[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_bn         │ (None, 48, 64, 64)     │            256 │ pool1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_relu       │ (None, 48, 64, 64)     │              0 │ conv2_block1_0_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 48, 64, 128)    │          8,192 │ conv2_block1_0_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 48, 64, 128)    │            512 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 48, 64, 128)    │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 48, 64, 32)     │         36,864 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_concat       │ (None, 48, 64, 96)     │              0 │ pool1[0][0],           │
│ (Concatenate)             │                        │                │ conv2_block1_2_conv[0… │
├──────────────────────

 Total params: 7,037,504 (26.85 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 7,037,504 (26.85 MB)

In [16]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)             │ (None, 6, 8, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,565,895 (28.86 MB)

 Trainable params: 528,391 (2.02 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [28]:
# train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    #class_weight=class_weights,
    validation_data=val_generator,
    validation_steps=val_steps,
    epochs=15, verbose=1)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


277/277 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - categorical_accuracy: 0.4373 - loss: 1.4920

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


277/277 ━━━━━━━━━━━━━━━━━━━━ 75s 200ms/step - categorical_accuracy: 0.4376 - loss: 1.4912 - val_categorical_accuracy: 0.8190 - val_loss: 0.5590
Epoch 2/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 47s 170ms/step - categorical_accuracy: 0.5867 - loss: 1.0781
Epoch 3/15
  2/277 ━━━━━━━━━━━━━━━━━━━━ 20s 73ms/step - categorical_accuracy: 0.6484 - loss: 0.9186

...

277/277 ━━━━━━━━━━━━━━━━━━━━ 38s 139ms/step - categorical_accuracy: 0.7124 - loss: 0.7756
Epoch 13/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 39s 143ms/step - categorical_accuracy: 0.7096 - loss: 0.7564 - val_categorical_accuracy: 0.8455 - val_loss: 0.4650
Epoch 14/15
277/277 ━━━━━━━━━━━━━━━━━━━━ 33s 121ms/step - categorical_accuracy: 0.7177 - loss: 0.7543
Epoch 15/15
  2/277 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - categorical_accuracy: 0.6562 - loss: 0.6774

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


 80/277 ━━━━━━━━━━━━━━━━━━━━ 30s 153ms/step - categorical_accuracy: 0.7269 - loss: 0.7087

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


277/277 ━━━━━━━━━━━━━━━━━━━━ 15s 54ms/step - categorical_accuracy: 0.7286 - loss: 0.7060 - val_categorical_accuracy: 0.8501 - val_loss: 0.4656


In [29]:
train_batches = datagen.flow_from_directory(train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=train_batch_size)


277.0